In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [152]:
import os
import keras.backend as K

from elmo.lm_generator import LMDataGenerator, MTLLMDataGenerator
from elmo.model_girnet import ELMo

## Note
- fasttext: sent_vector: unsup: [0.5040783036202435, 0.44986425566245447]
- fasttext: sent_vector: sup: [0.5628058728541675, 0.5187983749157842]
- fasttext: lstm: unsup [0.6182707995419401, 0.5768815131210775] 
- fasttext: lstm: sup [0.6705, 0.6624]
- lstm 400,200: preplexity - 14 on cm_p_test_1000.txt
- lstm - mtl preplexity: 14.398013495954116, loss: 1.6848554463386536
- girnet: perplexity: 165.677334, losss

In [153]:
DATA_SET_DIR = '../twiter_scrapping/data/'

In [154]:
parameters = {
    'name': 'cm_girnet_4',
    'multi_processing': True,
    'n_threads': 10,
    'cuDNN': True if len(K.tensorflow_backend._get_available_gpus()) else False,
    'train_dataset': 'cm_p_train_small.txt',
    'valid_dataset': 'cm_p_test_1000.txt',
    'test_dataset': 'cm_test_main_1000.txt',
    'vocab': 'vocab',
    'vocab_size': 525133,
    'num_sampled': 8000,
    'charset_size': 262,
    'sentence_maxlen': 32,
    'token_maxlen': 50,
    'token_encoding': 'word',
    'epochs': 1,
    'patience': 2,
    'batch_size': 32,
    'clip_value': 1,
    'cell_clip': 5,
    'proj_clip': 5,
    'lr': 0.2,
    'shuffle': True,
    'n_lstm_layers': 1,
    'n_highway_layers': 1,
    'cnn_filters': [[1, 32],
                    [2, 32],
                    [3, 64],
                    [4, 128],
                    [5, 256],
                    [6, 512],
                    [7, 512]
                    ],
    'lstm_units_size': 400,
    'hidden_units_size': 200,
    'char_embedding_size': 16,
    'dropout_rate': 0.1,
    'word_dropout_rate': 0.05,
    'weight_tying': True,
    'unidirectional': True,
    'subwords': False,
    'subword_weight': None,
    'filters': 200,
    'kernel_size': 3,
    'pool_length': 1
    
}



In [155]:
# train_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'en_p_train_small.txt'),os.path.join(DATA_SET_DIR, 'es_p_train_small.txt'),os.path.join(DATA_SET_DIR, 'cm_p_train_small.txt')],
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding']
# )
# val_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'en_p_test_1000.txt'),os.path.join(DATA_SET_DIR, 'es_p_test_1000.txt'),os.path.join(DATA_SET_DIR, 'cm_p_test_1000.txt')],
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding']
# )
test_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset'])],
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding']
)

In [82]:
# train_en_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'en_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

# train_es_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'es_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

# train_cm_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'cm_p_train_small.txt'),
#                                   os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                   sentence_maxlen=parameters['sentence_maxlen'],
#                                   token_maxlen=parameters['token_maxlen'],
#                                   batch_size=parameters['batch_size'],
#                                   shuffle=parameters['shuffle'],
#                                   token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])
# val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
#                                 os.path.join(DATA_SET_DIR, parameters['vocab']),
#                                 sentence_maxlen=parameters['sentence_maxlen'],
#                                 token_maxlen=parameters['token_maxlen'],
#                                 batch_size=parameters['batch_size'],
#                                 shuffle=parameters['shuffle'],
#                                 token_encoding=parameters['token_encoding'])

In [5]:
# Set-up Generators
train_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['train_dataset']),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

In [156]:
# Compile ELMo
elmo_model = ELMo(parameters)
elmo_model.compile_elmo(print_summary=False)

In [7]:
from keras.utils import plot_model
plot_model(elmo_model._model, to_file='model.png')

In [10]:
# for tr in [train_en_generator, train_es_generator, train_cm_generator]:
#     elmo_model.train(train_data=tr, valid_data=val_generator)
#  5.5218
# elmo_model.train(train_data=train_generator, valid_data=val_generator)

In [157]:
elmo_model.load_temp()

In [158]:
elmo_model.save(sampled_softmax=False, temp=False)

Language Model saved successfully


In [86]:
# cm_p_test_1000.txt
# vanilla :  46.75967671612614
# mtl: 39.42567465448776
# girnet: 165.677334
# cm_main_test_1000.txt
# vanilla :  73.61951568054178
# mtl: 58.723475872121696
# girnet: 132.512263
elmo_model.evaluate(test_generator)

Forward Langauge Model Perplexity: 46.54720741329034


In [ ]:
# Build ELMo meta-model to deploy for production and persist in disk
# elmo_model.wrap_multi_elmo_encoder(print_summary=True, save=True)

In [ ]:
# elmo_model.load()

In [ ]:
# Get ELMo embeddings to feed as inputs for downstream tasks
# elmo_embeddings = elmo_model.get_outputs(test_generator, output_type='word', state='mean')

# Generate 

In [159]:
import numpy as np
def sample_logits(preds, temperature=0.1):
    """

    Sample an index from a logit vector.

    :param preds:
    :param temperature:
    :return:
    """
    from scipy.misc import logsumexp
    preds = np.asarray(preds[0]).astype('float64')

    if temperature == 0.0:
        return np.argmax(preds)
    
    
    preds = preds / temperature
    preds = preds - logsumexp(preds)

    choice = np.random.choice(len(preds), 1, p=np.exp(preds))[0]

    return choice

In [160]:
vocab = test_generator.data2.vocab

In [163]:
id2word = {}
for key in vocab:
    id2word[vocab[key]] = key

In [164]:
def decode(sents_index):
    ans = ""
    for index in sents_index:
        ans = ans +  id2word[index] + " "
    return ans

In [148]:
def generate(seed, size=32):
    for i in range(len(seed), size):
        temp_seed_forward = np.array(seed[:-1]+[0]).reshape(len(seed),1,1)
        x_seed = np.array(seed).reshape(len(seed),1)
        pred = elmo_model._model.predict([x_seed, temp_seed_forward, temp_seed_forward])
        seed.append(sample_logits((pred[i-1])))
    return seed

In [176]:
# seed = [2, ]
# ans = generate(seed)
# decode(ans)

In [181]:
seed = [24]
for i in range(len(seed), size):
    temp_seed_forward = np.array(seed[:-1] + [0]).reshape(len(seed), 1, 1)
    x_seed = np.array(seed).reshape(len(seed), 1)
    x = [x_seed, temp_seed_forward, x_seed, temp_seed_forward, x_seed, temp_seed_forward]
    pred = elmo_model._model.predict(x)
    seed.append(sample_logits(pred[0][i - 1]))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.


In [182]:
decode(seed)

'i agru gasulla sincronica disfrutarais ungu malandra apaan reivindicaciã³n recuerdamelo restocked martinson celebrare clee commet mainframes '